In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import os
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
def load_dataset(dataset_path):
    images, labels = [], []
    
    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        
        if os.path.isdir(person_path):  # Chỉ lấy thư mục
            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                
                # Chỉ nhận file .jpg, .jpeg, .png
                if image_path.lower().endswith(('.jpg', '.jpeg', '.png')):  
                    img = cv2.imread(image_path)
                    img = cv2.resize(img, (100, 100))  # Resize về 100x100
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Chuyển sang RGB
                    images.append(img)
                    labels.append(person_name)
    
    return np.array(images), np.array(labels)

In [3]:
# Đường dẫn dataset (chỉnh lại nếu cần)
dataset_path = "../dataset"  # Nếu notebook nằm trong thư mục con
# Load dataset
X, y = load_dataset(dataset_path)
print(f"✅ Loaded {len(X)} images from {len(np.unique(y))} classes")
# ==========================

✅ Loaded 50 images from 4 classes


In [4]:
# 2️⃣ Xử lý dữ liệu
# ==========================

# Chuyển đổi nhãn thành số
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [5]:
# Chuẩn hóa ảnh về khoảng [0,1]
X = X / 255.0

In [6]:
# Chia dữ liệu train/test (80% train - 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [7]:
# 3️⃣ Xây dựng mô hình CNN
# ==========================
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Giảm overfitting

    layers.Dense(len(np.unique(y_encoded)), activation='softmax')  # Số lớp output bằng số người
])

# Compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
# 4️⃣ Huấn luyện mô hình
# ==========================
EPOCHS = 50
history = model.fit(X_train, y_train, epochs=EPOCHS, validation_data=(X_test, y_test))

Epoch 1/50
2/2 [==============================] - 1s 392ms/step - loss: 1.3435 - accuracy: 0.4500 - val_loss: 1.5590 - val_accuracy: 0.3000
Epoch 2/50
2/2 [==============================] - 0s 152ms/step - loss: 1.3159 - accuracy: 0.4000 - val_loss: 1.2784 - val_accuracy: 0.7000
Epoch 3/50
2/2 [==============================] - 0s 159ms/step - loss: 1.0458 - accuracy: 0.5250 - val_loss: 1.2249 - val_accuracy: 0.4000
Epoch 4/50
2/2 [==============================] - 0s 178ms/step - loss: 0.9960 - accuracy: 0.5750 - val_loss: 1.2329 - val_accuracy: 0.4000
Epoch 5/50
2/2 [==============================] - 0s 187ms/step - loss: 0.8924 - accuracy: 0.5750 - val_loss: 1.1029 - val_accuracy: 0.7000
Epoch 6/50
2/2 [==============================] - 0s 203ms/step - loss: 0.7577 - accuracy: 0.7250 - val_loss: 0.8917 - val_accuracy: 0.7000
Epoch 7/50
2/2 [==============================] - 0s 175ms/step - loss: 0.5660 - accuracy: 0.8000 - val_loss: 0.7609 - val_accuracy: 0.7000
Epoch 8/50
2/2 [====

In [ ]:
# 5️⃣ Lưu mô hình & label encoder
# ==========================
model.save("face_recognition_model.h5")

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("🎉 Đã lưu model và label encoder thành công!")

🎉 Đã lưu model và label encoder thành công!


: 